In [36]:
import pandas as pd 

In [37]:
def read_csv(file_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Đọc dữ liệu từ file CSV và trả về DataFrame.

    Parameters:
        file_path (str): Đường dẫn đến file CSV cần đọc.
        sep (str): Ký tự phân tách giữa các cột, mặc định là ','.Parameters

    Returns:
        pd.DataFrame: Dữ liệu được đọc vào từ file CSV.
    """
    try:
        df = pd.read_csv(file_path, sep=sep, encoding="utf-8")
        return df
    except FileNotFoundError:
        print(f" Không tìm thấy file tại: {file_path}")
    except Exception as e:
        print(f"Lỗi khi đọc file CSV: {e}")
    return pd.DataFrame()


def to_csv(df: pd.DataFrame, file_path: str) -> bool:
    """
    Lưu DataFrame thành file CSV sau khi làm sạch dữ liệu.

    Parameters:
        df (pd.DataFrame): DataFrame cần lưu.
        file_path (str): Đường dẫn lưu file CSV.

    Returns:
        bool: True nếu lưu thành công, False nếu gặp lỗi.
    """
    try:
        df.to_csv(file_path, index=False, encoding="utf-8")
        print(f"File đã được lưu tại: {file_path}")
        return True
    except FileNotFoundError:
        print(f"Không tìm thấy đường dẫn: {file_path}")
    except Exception as e:
        print(f"Lỗi khi lưu file CSV: {e}")
    return False

In [38]:
def extract_rating_features(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Tách điểm trung bình và số lượt đánh giá từ cột 'bus_rating'.

    Ví dụ:
        "4.7 (123)" -> rating_overall=4.7, reviewer_count=123

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột đánh giá.
        col_name (str): Tên cột chứa dữ liệu đánh giá, ví dụ 'bus_rating'.

    Returns:
        pd.DataFrame: Thêm 2 cột 'rating_overall' và 'reviewer_count'.
    """
    split_cols = df[col_name].str.split(' ', expand=True)
    df['rating_overall'] = pd.to_numeric(split_cols[0], errors='coerce')
    df['reviewer_count'] = split_cols[1].replace(r'[()]', '', regex=True).astype(int)
    return df

In [39]:
def extract_number_of_seats(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Trích xuất số lượng ghế từ cột 'seat_type' và tạo cột mới 'number_of_seat'.

    Ví dụ:
        "Xe giường nằm 40 chỗ"  ->  40
        "Ghế ngồi 29 chỗ"       ->  29

    Parameters:
        df (pd.DataFrame): DataFrame chứa cột 'seat_type' mô tả loại ghế.

    Returns:
        pd.DataFrame: DataFrame đã được thêm cột 'number_of_seat'.
    """

    # Trích xuất số
    df['number_of_seat'] = df[col_name].replace(r'[^0-9]', '', regex=True)
    df['number_of_seat'] = pd.to_numeric(df['number_of_seat'], errors='coerce').astype(int)

    return df

In [40]:
def normalize_fare(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Chuẩn hoá giá vé về dạng số nguyên (VND).

    Ví dụ:
        "Từ 350.000đ" -> 350000
        "200.000đ"    -> 200000

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu giá.
        cols (list[str]): Danh sách 2 cột ['original_price', 'sale_price'].

    Returns:
        pd.DataFrame: Thêm 2 cột mới 'price_original' và 'price_discounted'.
    """
    
    price = df[cols].replace(r'[^0-9]', '', regex=True)
    # fillna = 0 và ép kiểu sang int
    price = price.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)   

    df['price_original'] = price.iloc[:, 0] * 1000
    df['price_discounted'] = price.iloc[:, 1]

    return df


In [41]:
def convert_to_time(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuyển đổi cột thời gian từ chuỗi (string) sang kiểu datetime.time.

    Ví dụ:
        "07:30"  ->  "7:30:00"
        "21:05"  ->  "21:05:00"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu thời gian (ví dụ 'departure_time').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuyển sang kiểu datetime.time.
    """

    df[col_name] = pd.to_datetime(df[col_name], format='%H:%M', errors='coerce').dt.time
    return df

In [42]:
def normalize_date(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột ngày tháng từ chuỗi sang định dạng 'YYYY-MM-DD'.

    Ví dụ:
        "T4, 17/10/2025"  ->  "2025-10-17"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu ngày tháng (ví dụ 'departure_date').

    Returns:
        pd.DataFrame: DataFrame với cột 'departure_time' đã được chuẩn hóa.
    """

    date = df[col_name].str.split(', ', expand=True)[1]
    df[col_name] = pd.to_datetime(date, format='%d/%m/%Y', errors='coerce').dt.strftime('%Y-%m-%d')
    return df


In [43]:
def normalize_duration(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa cột thời lượng di chuyển sang tổng số phút (int).

    Ví dụ:
        "2h30m" -> 150
        "1h"    -> 60
        "45m"   -> 45

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa thời lượng di chuyển (ví dụ 'duration').

    Returns:
        pd.DataFrame: DataFrame với cột mới 'duration_m' biểu thị tổng số phút di chuyển.
    """

    # Trích xuất phần giờ và phút
    hours = df[col_name].str.extract(r'(\d+)h')[0].astype('Int64').fillna(0)
    mins = df[col_name].str.extract(r'(\d+)m')[0].astype('Int64').fillna(0)

    # Tính tổng số phút
    df['duration_m'] = hours * 60 + mins
    return df


In [44]:
def handle_bus_name(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Chuẩn hóa tên nhà xe bằng cách loại bỏ phần trong ngoặc.

    Ví dụ:
        "Hoàng Long (Ghế ngồi)" -> "Hoàng Long"

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.

    Returns:
        pd.DataFrame: DataFrame với cột 'bus_name' đã được làm sạch.
    """

    df[column].replace(r'\s*\([^)]+\)', '', regex=True, inplace=True)
    return df

In [45]:
def normalize_location_type(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    Chuẩn hóa loại địa điểm (ví dụ: 'Bến xe', 'Văn phòng', 'Other') 
    trong cột chỉ định của DataFrame bằng cách mở rộng các từ viết tắt thông dụng.

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu cần xử lý.
        col_name (str): Tên cột chứa dữ liệu văn bản cần chuẩn hóa
                        (ví dụ: 'pick_up_point' hoặc 'drop_off_point').

    Returns:
        pd.DataFrame: DataFrame với cột đã được chuẩn hóa loại địa điểm.
    """
    new_col = f"{col_name}_type"
    df[new_col] = df[col_name].str.lower().apply(
        lambda x: "Bến xe" if "bx" in x or "bến xe" in x
        else "Văn phòng" if "vp" in x or "văn phòng" in x
        else "Other"
    )

    return df

In [46]:
def fill_missing_ratings(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Điền giá trị thiếu cho các cột đánh giá theo trung bình của từng nhà xe.
    Nếu nhà xe không có dữ liệu, dùng trung bình toàn cột để thay thế.

    Returns:
        pd.DataFrame: DataFrame với các cột đánh giá đã được chuẩn hóa.
    """

    df[cols] = (
        df.groupby('company_name', group_keys=False)[cols]
          .apply(lambda g: g.fillna(g.mean().round(1)))
          .fillna(df[cols].mean().round(1))
    )

    return df

In [47]:
def drop_unused_columns(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Loại bỏ các cột không cần thiết khỏi DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame cần xử lý.
        columns_to_drop (list[str]): Danh sách tên cột cần xóa.

    Returns:
        pd.DataFrame: DataFrame sau khi loại bỏ các cột không cần thiết.
    """

    # Loại bỏ các cột đã được trích xuất dữ liệu hoàn thành, không dùng tới nữa
    df.drop(columns=cols, errors='ignore', inplace=True)

    # Loại bỏ cột có toàn bộ giá trị là None (nếu có)
    df.dropna(axis=1, how='all', inplace=True)

    # Lọc dữ liệu có giá vé hoặc số người đánh giá > 0
    df = df[(df['reviewer_count'] > 0) & (df['price_original'] > 0)]
    return df

In [48]:
def rename_cols(df: pd.DataFrame, dict_rename_cols: dict[str]):

    df.rename(columns=dict_rename_cols, inplace=True)
    return df

In [49]:
def preprocess_transport_data(df, dict_rename_cols, unused_cols, rating_cols):
    """ 
    Tiền xử lý toàn bộ dữ liệu xe khách:
    - Chuẩn hoá rating, giá, thời gian, tên nhà xe
    - Làm sạch điểm đón/trả
    - Điền giá trị thiếu cho rating theo trung bình của từng nhà xe
    """

    df = extract_rating_features(df, 'bus_rating')  # trích xuất phần đánh giá (sao)
    df = extract_number_of_seats(df, 'seat_type')   # trích xuất ra số lượng ghế
    df = normalize_fare(df, ['price_original', 'price_discounted'])   # chuẩn hoá, ép kiểu dữ liệu sang int
    df = convert_to_time(df, 'departure_time')  # chuẩn hoá thành hh:mm:00
    df = normalize_date(df, 'departure_date')   # chuẩn hoá thành yyyy-MM-DD
    df = normalize_duration(df, 'duration') # chuẩn hoá thành phút
    df = handle_bus_name(df, 'company_name')    # Làm sạch tên nhà xe
    df = normalize_location_type(df, 'pickup_point')
    df = normalize_location_type(df, 'dropoff_point')
    df = fill_missing_ratings(df, rating_cols)  # fillna()
    df = rename_cols(df, dict_rename_cols)
    df = drop_unused_columns(df, unused_cols)   # bỏ những cột không dùng đến/đã được trích xuất
    return df

---

In [50]:
dict_rename_cols = {
# ratings
    'An toàn': 'rating_safety',
    'Thông tin chính xác': 'rating_info_accuracy',
    'Thông tin đầy đủ': 'rating_info_completeness',
    'Thái độ nhân viên': 'rating_staff_attitude',
    'Tiện nghi & thoải mái': 'rating_comfort',
    'Chất lượng dịch vụ': 'rating_service_quanlity',
    'Đúng giờ': 'rating_punctuality',
# trips info
    'pick_up_point_type': 'pickup_point',
    'drop_of_point_type': 'dropoff_point'
}

rating_cols = ['An toàn', 'Thông tin chính xác', 'Thông tin đầy đủ','Thái độ nhân viên', 'Tiện nghi & thoải mái','Chất lượng dịch vụ', 'Đúng giờ']
unused_cols = ['arrival_time', 'arrival_date', 'bus_rating', 'seat_type', 'sale_price', 'original_price', 'percent_discount', 'duration', 'pickup_point', 'dropoff_point']

In [58]:
df = pd.read_csv('../data/raw/Sài Gòn_Vũng Tàu_1_11_2025.csv')
df

,company_name,bus_rating,seat_type,departure_time,pickup_point,arrival_date,arrival_time,dropoff_point,duration,price_original,...,departure_date,start_point,destination,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Vie Limousine,4.6 (4755),Limousine 9 chỗ,09:15,• Văn phòng Quận 1,NaN,11:15,• Văn phòng Vũng Tàu,2h,190.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Anh Quốc Limousine,4.8 (3548),Limousine 9 chỗ,12:00,• Sân Bay Tân Sơn Nhất,NaN,14:30,• Văn Phòng Vũng Tàu,2h30m,190.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,4.9,4.8,4.9,4.9,4.8,4.8,4.9
2,Hoa Mai,4.4 (4314),Limousine 9 chỗ,16:30,• Văn phòng quận 1,NaN,17:55,• Bến xe Bà Rịa,1h25m,200.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bến Thành Travel,4.6 (594),Limousine 9 chỗ,08:15,• Văn phòng Sài Gòn,NaN,10:15,• Văn phòng Vũng Tàu,2h,190.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Toàn Thắng - Vũng Tàu,4.6 (453),Limousine 9 chỗ,14:30,• Sân bay Tân Sơn Nhất,NaN,17:30,• Vũng Tàu,3h,190.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,4.6,4.6,4.6,4.6,4.5,4.5,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Anh Quốc Limousine,4.8 (3548),Limousine 9 chỗ,19:31,• Văn phòng Quận 1,NaN,21:46,• Văn Phòng Vũng Tàu,2h15m,180.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,Vie Limousine,4.6 (4755),Limousine 9 chỗ,08:40,• Hàng Xanh Khu Du Lịch Văn Thánh,NaN,11:00,• Văn phòng Vũng Tàu,2h20m,210.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,4.7,4.7,4.7,4.7,4.7,4.7,4.8
117,Anh Quốc Limousine,4.8 (3548),Limousine 9 chỗ,14:10,• Văn Phòng Nguyễn Văn Trỗi,NaN,16:30,• Văn Phòng Vũng Tàu,2h20m,190.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,Hoa Mai,4.4 (4314),Limousine 9 chỗ,10:00,• Văn phòng quận 5,NaN,12:20,• Nội thành Vũng Tàu,2h20m,200.0,...,"T7, 01/11/2025",Sài Gòn,Bà Rịa-Vũng Tàu,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
df = preprocess_transport_data(df, dict_rename_cols, unused_cols, rating_cols)
# ../data/raw/Sài Gòn_Vũng Tàu_1_11_2025.csv
# df.drop_duplicates(keep='first', inplace=True)

In [60]:
df.duplicated().sum()

0

In [61]:
df

,company_name,departure_time,price_original,price_discounted,departure_date,start_point,destination,rating_safety,rating_info_accuracy,rating_info_completeness,rating_staff_attitude,rating_comfort,rating_service_quanlity,rating_punctuality,rating_overall,reviewer_count,number_of_seat,duration_m,pickup_point_type,dropoff_point_type
0,Vie Limousine,09:15:00,190000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.7,4.7,4.7,4.7,4.7,4.7,4.8,4.6,4755,9,120,Văn phòng,Văn phòng
1,Anh Quốc Limousine,12:00:00,190000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.9,4.8,4.9,4.9,4.8,4.8,4.9,4.8,3548,9,150,Other,Văn phòng
2,Hoa Mai,16:30:00,200000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.5,4.5,4.5,4.4,4.4,4.3,4.7,4.4,4314,9,85,Văn phòng,Bến xe
3,Bến Thành Travel,08:15:00,190000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.7,4.7,4.7,4.5,4.6,4.5,4.8,4.6,594,9,120,Văn phòng,Văn phòng
4,Toàn Thắng - Vũng Tàu,14:30:00,190000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.6,4.6,4.6,4.6,4.5,4.5,4.9,4.6,453,9,180,Other,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Anh Quốc Limousine,19:31:00,180000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.9,4.8,4.9,4.9,4.8,4.8,4.9,4.8,3548,9,135,Văn phòng,Văn phòng
116,Vie Limousine,08:40:00,210000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.7,4.7,4.7,4.7,4.7,4.7,4.8,4.6,4755,9,140,Other,Văn phòng
117,Anh Quốc Limousine,14:10:00,190000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.9,4.8,4.9,4.9,4.8,4.8,4.9,4.8,3548,9,140,Văn phòng,Văn phòng
118,Hoa Mai,10:00:00,200000,0,2025-11-01,Sài Gòn,Bà Rịa-Vũng Tàu,4.5,4.5,4.5,4.4,4.4,4.3,4.7,4.4,4314,9,140,Văn phòng,Other


In [55]:
df.isna().sum()

company_name                0
departure_time              0
price_original              0
price_discounted            0
departure_date              0
start_point                 0
destination                 0
rating_safety               0
rating_info_accuracy        0
rating_info_completeness    0
rating_staff_attitude       0
rating_comfort              0
rating_service_quanlity     0
rating_punctuality          0
rating_overall              0
reviewer_count              0
number_of_seat              0
duration_m                  0
pickup_point_type           0
dropoff_point_type          0
dtype: int64